In [ ]:
%load_ext autoreload
%autoreload 2

import json
import os
import pandas as pd
from IPython.display import Image
from matplotlib import pyplot as plt
import cv2

from research.utils.data_access_utils import S3AccessUtils

import sys
sys.path.append('..')
from simple_object_orientation.utils import *
from sift_utils.point_cloud import generate_point_cloud

s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

In [ ]:
f, _, _ = s3.download_from_url('https://aquabyte-crops.s3.eu-west-1.amazonaws.com/environment=production/site-id=96/pen-id=159/date=2021-04-16/hour=13/at=2021-04-16T13:50:24.060390000Z/left_frame_crop_0_1345_4096_3000.jpg')



<h1> Utility Methods </h1>

In [ ]:
def get_spatial_attributes(annotation: dict, camera_metadata: CameraMetadata) -> dict:
    """Generate location and orientation attributes of fish key-points."""

    X_left, X_right = get_2D_coords_from_ann(annotation)
    X = get_3D_coords_from_2D(X_left, X_right, camera_metadata)

    localized_X = center_3D_coordinates(X)
    B = compute_orthonormal_basis(localized_X)
    local_yaw, local_pitch, local_roll = rotation_matrix_to_euler_angles(B)

    B = compute_orthonormal_basis(X)
    global_yaw, global_pitch, global_roll = rotation_matrix_to_euler_angles(B)

    spatial_attrs = dict(
        centroid_x=np.mean(X[:, 0]),
        centroid_y=np.mean(X[:, 1]),
        centroid_z=np.mean(X[:, 2]),
        local_yaw=local_yaw,
        local_pitch=local_pitch,
        local_roll=local_roll,
        global_yaw=global_yaw,
        global_pitch=global_pitch,
        global_roll=global_roll
    )

    return spatial_attrs

def compute_orthonormal_basis(coords):
    """Given a set of fish 3D key-point coordinates, approximate the orthonormal basis
    corresponding to the new coordinate system that is axis-aligned with the coordinates."""

    tail_notch_idx = BODY_PARTS.index('TAIL_NOTCH')
    upper_lip_idx = BODY_PARTS.index('UPPER_LIP')
    dorsal_fin_idx = BODY_PARTS.index('DORSAL_FIN')
    pelvic_fin_idx = BODY_PARTS.index('PELVIC_FIN')

    u = coords[upper_lip_idx] - coords[tail_notch_idx]  # vector from TAIL_NOTCH to UPPER_LIP
    w = coords[dorsal_fin_idx] - coords[pelvic_fin_idx]  # vector from PELVIC_FIN to DORSAL_FIN
    u = u / np.linalg.norm(u)
    w = w - (np.dot(u, w)) * u
    w = w / np.linalg.norm(w)
    v = np.cross(w, u)
    v = v / np.linalg.norm(v)
    return np.vstack((u, v, w)).T

def reflect(coords, do_reflection):
    if do_reflection:
        return coords
    else:
        new_coords = coords[:]
        new_coords[:, 0] = -new_coords[:, 0]
        return new_coords


<h1> Load GTSF data and get lateral fish for each fish ID </h1>

In [ ]:
s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
trainig_dataset_url = 'https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/alok/training_datasets/2021-05-29_weight_estimation_training_dataset_15367.csv'
training_dataset_f, _, _ = s3.download_from_url(trainig_dataset_url)
df = pd.read_csv(training_dataset_f)

In [ ]:
local_yaws, local_pitches, local_rolls, X_list = [], [], [], []
count = 0
for idx, row in df.iterrows():
    ann = json.loads(row.keypoints.replace("'", '"'))
    camera_metadata = json.loads(row.camera_metadata.replace("'", '"'))
    cm = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )
    
    X_left, X_right = get_2D_coords_from_ann(ann)
    X = get_3D_coords_from_2D(X_left, X_right, cm)
    X_list.append(X)
    
    spatial_attrs = get_spatial_attributes(ann, cm)
    local_yaws.append(spatial_attrs['local_yaw'])
    local_pitches.append(spatial_attrs['local_pitch'])
    local_rolls.append(spatial_attrs['local_roll'])
    
    if count % 1000 == 0:
        print(count)
    count += 1
    
df['X'] = X_list
df['local_yaw'] = np.array(local_yaws) * 180 / np.pi
df['local_pitch'] = np.array(local_pitches) * 180 / np.pi
df['local_roll'] = np.array(local_rolls) * 180 / np.pi
df['primary_key'] = range(df.shape[0])

In [ ]:
primary_keys = []
for fish_id in df.fish_id.unique():
    mask = (df.fish_id == fish_id) & ((df.local_yaw % 180).abs() < 10) & (df.local_pitch.abs() < 5) & (df.local_roll.abs() < 5)
    if mask.sum():
        key = df[mask].sort_values('captured_at', ascending=True).primary_key.iloc[0]
        primary_keys.append(key)
        
key_mask = df.primary_key.isin(primary_keys)

<h1> Generate Point Cloud </h1>

In [ ]:
def get_bbox_coords(ann, left_image_arr, right_image_arr, buffer=50):
    x_min = max(min([item['xCrop'] for item in ann['leftCrop']]) - buffer, 0)
    y_min = max(min([item['yCrop'] for item in ann['leftCrop']]) - buffer, 0)
    x_max = min(max([item['xCrop'] for item in ann['leftCrop']]) + buffer, left_image_arr.shape[1])
    y_max = min(max([item['yCrop'] for item in ann['leftCrop']]) + buffer, right_image_arr.shape[0])

    
    left_bbox = [x_min, y_min, x_max, y_max]
    
    x_min = max(min([item['xCrop'] for item in ann['rightCrop']]) - buffer, 0)
    y_min = max(min([item['yCrop'] for item in ann['rightCrop']]) - buffer, 0)
    x_max = min(max([item['xCrop'] for item in ann['rightCrop']]) + buffer, right_image_arr.shape[1])
    y_max = min(max([item['yCrop'] for item in ann['rightCrop']]) + buffer, right_image_arr.shape[0])
    
    right_bbox = [x_min, y_min, x_max, y_max]
    
    left_bbox_x_frame_min = max(min([item['xFrame'] for item in ann['leftCrop']]) - buffer, 0)
    left_bbox_y_frame_min = max(min([item['yFrame'] for item in ann['leftCrop']]) - buffer, 0)
    right_bbox_x_frame_min = max(min([item['xFrame'] for item in ann['rightCrop']]) - buffer, 0)
    right_bbox_y_frame_min = max(min([item['yFrame'] for item in ann['rightCrop']]) - buffer, 0)
    
    new_ann = {'leftCrop': [], 'rightCrop': []}
    for item in ann['leftCrop']:
        new_item = {}
        new_item['keypointType'] = item['keypointType']
        new_item['xFrame'] = item['xFrame']
        new_item['yFrame'] = item['yFrame']
        new_item['xCrop'] = item['xFrame'] - left_bbox_x_frame_min
        new_item['yCrop'] = item['yFrame'] - left_bbox_y_frame_min
        new_ann['leftCrop'].append(new_item)
        
    for item in ann['rightCrop']:
        new_item = {}
        new_item['keypointType'] = item['keypointType']
        new_item['xFrame'] = item['xFrame']
        new_item['yFrame'] = item['yFrame']
        new_item['xCrop'] = item['xFrame'] - right_bbox_x_frame_min
        new_item['yCrop'] = item['yFrame'] - right_bbox_y_frame_min
        new_ann['rightCrop'].append(new_item)
    
    return left_bbox, right_bbox, new_ann
    
    

In [ ]:
idx = 66
row = df[key_mask].iloc[idx]
left_image_url = row.left_image_url
right_image_url = row.right_image_url
left_image_f, _, _ = s3.download_from_url(left_image_url)
right_image_f, _, _ = s3.download_from_url(right_image_url)
left_image_arr = cv2.cvtColor(cv2.imread(left_image_f), cv2.COLOR_BGR2RGB)
right_image_arr = cv2.cvtColor(cv2.imread(right_image_f), cv2.COLOR_BGR2RGB)
left_bbox, right_bbox, ann = \
    get_bbox_coords(json.loads(row.keypoints.replace("'", '"')), left_image_arr, right_image_arr)
left_image_arr = left_image_arr[left_bbox[1]:left_bbox[3], left_bbox[0]:left_bbox[2]]
right_image_arr = right_image_arr[right_bbox[1]:right_bbox[3], right_bbox[0]:right_bbox[2]]



In [ ]:
left_points, right_points, left_corner, right_corner = generate_point_cloud(ann, left_image_arr, right_image_arr)
point_cloud = get_3D_coords_from_2D(left_points, right_points, cm)
rgbd = np.array([left_points[:, 0] - left_corner[0], left_points[:, 1] - left_corner[1], point_cloud[:, 1]]).T


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from scipy.spatial import Delaunay

fig1, ax1 = plt.subplots(figsize=(20, 10))

points = rgbd #[np.random.choice(rgbd.shape[0], 100)]
triang = tri.Triangulation(points[:, 0], points[:, 1])
ax1.set_aspect('equal')
tpc = ax1.tripcolor(triang, points[:, 2], shading='flat')
fig1.colorbar(tpc)
plt.imshow(left_image_arr)
plt.gca().set_aspect('equal', adjustable='box')
ax1.set_title('tripcolor of Delaunay triangulation, flat shading')
